## Question 1 (a)

Import moudles

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import layers

準備資料集

In [ ]:
train_data = pd.read_csv("HW1-1.csv")
train_data.head()

,x,y
0,-0.054504,-0.223653
1,0.724703,-0.363040
2,0.269098,-0.383166
3,-0.258074,-0.154802
4,-0.311265,-0.148746


In [ ]:
train_features = train_data.copy()
train_labels   = train_features.pop('y')
train_features = np.array(train_features)

建構模型

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(32),
    layers.Dense(16),
    layers.Dense(1)
])

model.compile(loss = tf.losses.MeanSquaredError(),
              optimizer = tf.optimizers.Adam())

訓練

In [ ]:
model.fit(train_features, train_labels, epochs=10, verbose=0)

結果

In [ ]:
model.eva

## Question 1 (b)

準備資料集

In [16]:
ORDER = 32

# Prepare data
dataframe = pd.read_csv("HW1-1.csv", index_col=None, dtype=np.float32)
dataframe.rename(columns={"x":"x_1", "y":"y"}, inplace=True)

# Calculate x to x^n, x^n-1, ..., x^1
for n in range(2, ORDER+1):
    dataframe[f"x_{n}"] = dataframe[f"x_{n-1}"] * dataframe["x_1"]

dataframe = dataframe.reindex(columns=[f"x_{n}" for n in range(1, ORDER+1)]+['y'])

training_set = dataframe.sample(frac=0.8, random_state=200)
test_set = dataframe.drop(training_set.index)

建構模型

In [17]:
model = tf.keras.Sequential([
    layers.Input(shape=(ORDER,)),
    layers.Dense(1),
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.MeanSquaredError()
)

訓練

In [18]:
MAX_NUMBER_OF_EPOCH=10
for i in range(MAX_NUMBER_OF_EPOCH):
    # Fit
    model.fit(training_set.drop('y', axis=1), training_set['y'], epochs=20, verbose=0)

    # Evaluate model
    test_loss = model.evaluate(x=test_set.drop('y', axis=1), y=test_set['y'])
    if test_loss < 0.1:
        break

assert test_loss < 0.1

1/1 [==============================] - 0s 20ms/step - loss: 0.0700


結果

In [21]:
test_loss = model.evaluate(x=test_set.drop('y', axis=1), y=test_set['y'])
print("Test loss: ", test_loss)

1/1 [==============================] - 0s 24ms/step - loss: 0.0700
Test loss:  0.06996189802885056


In [22]:
print("i. Print out the coefficients of the equation.")
kernel, biases = model.get_layer(index=0).weights
print("Weight: ", end='')
print(kernel.value().numpy())
print("Bias: ", end='')
print(sum(biases.value().numpy()))

i. Print out the coefficients of the equation.
Weight: [[ 0.1007423 ]
 [-0.06297268]
 [-0.02943124]
 [-0.23155521]
 [-0.2261115 ]
 [ 0.0561245 ]
 [-0.09686817]
 [ 0.3277167 ]
 [-0.15720591]
 [-0.25675574]
 [-0.24861994]
 [-0.05378663]
 [-0.10524174]
 [-0.18138628]
 [ 0.25936326]
 [-0.10369682]
 [-0.16191235]
 [ 0.34122032]
 [-0.08900369]
 [-0.10875673]
 [-0.12178234]
 [-0.1915558 ]
 [ 0.115853  ]
 [-0.0097118 ]
 [ 0.48860645]
 [ 0.33386782]
 [-0.11956736]
 [ 0.20520154]
 [ 0.01304738]
 [-0.28678012]
 [ 0.3048031 ]
 [ 0.10466629]]
Bias: -0.12550900876522064
ii. Calculate the following gradients:


In [34]:
def generate_w_power_array(value):
    arr = np.ndarray(shape=(ORDER, 1), dtype=np.float32)
    arr[0] = value
    for i in range(1, ORDER):
        arr[i] = arr[i] * arr[0]

    return arr.reshape((1, ORDER))

print("ii. Calculate the following gradients:")

ii. Calculate the following gradients:


LookupError: No gradient defined for operation'IteratorGetNext' (op type: IteratorGetNext). In general every operation must have an associated `@tf.RegisterGradient` for correct autodiff, which this op is lacking. If you want to pretend this operation is a constant in your program, you may insert `tf.stop_gradient`. This can be useful to silence the error in cases where you know gradients are not needed, e.g. the forward pass of tf.custom_gradient. Please see more details in https://www.tensorflow.org/api_docs/python/tf/custom_gradient.